In [2]:
import numpy as np
import pandas as pd

In [3]:
!pip install tensorflow
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ImportError: ignored

In [ ]:
dataset = pd.read_csv('hepsiburada.csv')

In [ ]:
dataset

In [ ]:
target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()

In [ ]:
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [ ]:
x_train[500]

In [ ]:
x_train[800]

In [ ]:
y_train[800]

In [ ]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
tokenizer.fit_on_texts(data)

In [ ]:
tokenizer.word_index

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)

In [ ]:
x_train[800]

In [ ]:
print(x_train_tokens[800])

In [ ]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [ ]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [ ]:
np.mean(num_tokens)

In [ ]:
np.max(num_tokens)

In [ ]:
np.argmax(num_tokens)

In [ ]:
x_train[21941]

In [ ]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

In [ ]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

In [ ]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)

In [ ]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [ ]:
x_train_pad.shape

In [ ]:
x_test_pad.shape

In [ ]:
np.array(x_train_tokens[800])

In [ ]:
x_train_pad[800]

In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [ ]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [ ]:
x_train[800]

In [ ]:
tokens_to_string(x_train_tokens[800])

In [ ]:
model = Sequential()

In [ ]:
embedding_size = 50

In [ ]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [ ]:
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
optimizer = Adam(lr=1e-3)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train_pad, y_train, epochs=5, batch_size=256)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
result = model.evaluate(x_test_pad, y_test)

In [ ]:
result[1]

In [ ]:
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

In [ ]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [ ]:
cls_true = np.array(y_test[0:1000])

In [ ]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [ ]:
len(incorrect)

In [ ]:
idx = incorrect[0]
idx

In [ ]:
text = x_test[idx]
text

In [ ]:
y_pred[idx]

In [ ]:
cls_true[idx]

In [ ]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [ ]:
tokens = tokenizer.texts_to_sequences(texts)

In [ ]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

In [ ]:
model.predict(tokens_pad)